In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://github.com/allenai/S2AFF/raw/main/data/gold_affiliation_annotations.csv')

In [3]:
data = []
for i, row in df.iterrows():
    elt = {}
    elt['label'] = row.original_affiliation
    elt['split'] = row.split
    rors = []
    for r in eval(row.labels):
        if 'ror.org' in r:
            rors.append(r.replace('https://ror.org/', ''))
    elt['ror'] = rors
    if rors:
        data.append(elt)

In [29]:
match({'query': "Institute for Theoretical Physics, State University of New York at Stony Brook, Stony Brook, Long Island, NY 11794, USA", 'year':'2020', 
                         'strategies': strategies,
                         'type': 'ror', 'index_prefix': 'matcher'})

{'highlights': {'ror_name;ror_supervisor_name;ror_city;ror_country': {'05qghxh33': {'ror_city': [['institute for theoretical physics, state university of new york at <em>stony</em> <em>brook</em>, <em>stony</em> <em>brook</em>, long island']],
    'ror_country': [['theoretical physics, state university of new york at stony brook, stony brook, long island, ny 11794, <em>usa</em>']],
    'ror_name': [['institute theoretical physics, state university new york at <em>stony</em> <em>brook</em>, <em>stony</em> <em>brook</em>, long island, ny']],
    'ror_supervisor_name': [['institute theoretical physics, <em>state</em> <em>university</em> <em>new</em> <em>york</em> at stony brook, stony brook, long island, ny']]}}},
 'other_ids': [],
 'results': ['05qghxh33'],
 'version': '0.1.5'}

In [24]:
import requests, json

strategies = [
    [['ror_id']],
    [['ror_grid_id']],
    [['ror_name', 'ror_supervisor_name', 'ror_acronym', 'ror_city_nuts_level2', 'ror_country'],
    # ['ror_name', 'ror_supervisor_name', 'ror_acronym', 'ror_city_nuts_level2', 'ror_country_code']
    ],
    [['ror_name', 'ror_acronym', 'ror_city', 'ror_country'],
  #   ['ror_name', 'ror_acronym', 'ror_city', 'ror_country_code']
    ],
    [['ror_name', 'ror_supervisor_name', 'ror_city', 'ror_country'],
    # ['ror_name', 'ror_supervisor_name', 'ror_city', 'ror_country_code']
    ],
       [['ror_name', 'ror_supervisor_name', 'ror_city_nuts_level2', 'ror_country'],
   #  ['ror_name', 'ror_supervisor_name', 'ror_city_nuts_level2', 'ror_country_code']
       ],
    [
        ['ror_name', 'ror_city', 'ror_country'], 
        #['ror_name', 'ror_city', 'ror_country_code'],
    ],
    #[['ror_acronym', 'ror_city', 'ror_country'], ['ror_acronym', 'ror_city', 'ror_country_code']],
    [['ror_name', 'ror_acronym', 'ror_city'], ['ror_name', 'ror_acronym', 'ror_country'],
    # ['ror_name', 'ror_acronym', 'ror_country_code']
    ],
    [['ror_name', 'ror_country'], 
    # ['ror_name', 'ror_country_code']
    ],
    [['ror_name', 'ror_city'], ['ror_name', 'ror_city_nuts_level2']]
#    ,[['ror_name_unique', 'ror_city_nuts_level2']],
#    [['ror_acronym_unique', 'ror_city_nuts_level2']],
#    [['ror_name_unique', 'ror_country']],
#    [['ror_acronym_unique', 'ror_country']]
]

def match(conditions):
    return requests.post("http://localhost:5004/match", json=conditions).json()


def compute_precision_recall(data, split = None):

    index_prefix = 'matcher'
    local = True
    match_type = 'ror'

    nb_TP, nb_FP, nb_FN = 0, 0, 0
    false_positive, false_negative = [], []
    for ix, d in enumerate(data):
        
        if split and d['split'] != split:
            continue
        
        if ix%100==0:
            print(ix, end=',')
        if d.get(match_type):
            res = match({'query': d['label'], 'year':'2020', 
                         'strategies': strategies,
                         'type': match_type, 'index_prefix': index_prefix})
            for x in res['results']:
                if x in d[match_type]:
                    nb_TP += 1
                else:
                    nb_FP += 1
                    false_positive.append(d)
            for x in d[match_type]:
                if x not in res['results']:
                    nb_FN += 1
                    false_negative.append(d)

    precision = nb_TP / (nb_TP + nb_FP)
    recall = nb_TP / (nb_TP + nb_FN)
    res = {'precision' : precision, 'recall' : recall}
    return {'res': res, 'false_positive': false_positive, 'false_negative': false_negative} 


In [25]:
res = compute_precision_recall(data)
res['res']

0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,

{'precision': 0.9002447980416156, 'recall': 0.6270247229326513}

In [30]:
res_test = compute_precision_recall(data, 'test')
res_test['res']

300,500,800,1000,1500,1700,1900,

{'precision': 0.9015748031496063, 'recall': 0.7351524879614767}

In [31]:
res_test['false_positive']

[{'label': 'Science and Technology on Thermostructural Composite Materials Laboratory, Northwestern Polytechnical University, Xi’an, Shaanxi 710072, PR China',
  'split': 'test',
  'ror': ['01y0j0j86']},
 {'label': 'National Key Laboratory of Thermostructure Composite Materials, School of Materials Science, Northwestern Polytechnical University, Xi’an Shaanxi 710072, PR China',
  'split': 'test',
  'ror': ['01y0j0j86']},
 {'label': 'National Key Laboratory of Thermostructure Composite Materials, School of Materials Science, Northwestern Polytechnical University, Xi’an, Shannxi 710072, PR China',
  'split': 'test',
  'ror': ['01y0j0j86']},
 {'label': 'Science and Technology on Thermostructure Composite Materials Laboratory, Northwestern Polytechnical University, Xi’an, Shaanxi 710072, China',
  'split': 'test',
  'ror': ['01y0j0j86']},
 {'label': 'Inst. of Software, Peking Univ., Beijing, China',
  'split': 'test',
  'ror': ['02v51f717']},
 {'label': 'Key Laboratory for Environmental an

In [20]:
res['false_positive'][0]

{'label': 'The Laboratory of Photochemistry, Center for Molecular Science, Institute of Chemistry, Chinese Academy of Sciences, Beijing 100080, PR China',
 'split': 'train',
 'ror': ['034t30j35']}

In [ ]:
# The affiliation-matcher also gives 048y1rc66 which seems ok...

In [21]:
res_test['false_positive']

[{'label': 'Inst. of Software, Peking Univ., Beijing, China',
  'split': 'test',
  'ror': ['02v51f717']},
 {'label': 'Illinois Genetic Algorithms Laboratory, University of Illinois, Urbana, IL#TAB#',
  'split': 'test',
  'ror': ['047426m28']},
 {'label': 'Department of General Engineering University of Illinois, Urbana, IL 61801, USA#TAB#',
  'split': 'test',
  'ror': ['047426m28']},
 {'label': 'Department of General Engineering, Illinois Genetic Algorithms Laboratory, University of Illinois, Urbana, IL 61801, USA#TAB#',
  'split': 'test',
  'ror': ['047426m28']},
 {'label': 'Illinois Genetic Algorithms Laboratory University of Illinois Urbana, IL 61801 USA deg@illigal.ge.uiuc.edu#TAB#',
  'split': 'test',
  'ror': ['047426m28']},
 {'label': 'Department of Applied Mathematics and Theoretical Physics, Cambridge University, Wilberforce Road, CB3 0WA Cambridge, UK',
  'split': 'test',
  'ror': ['013meh722']},
 {'label': 'Department of Applied Mathematics and Theoretical Physics, Centre fo

In [10]:
r = requests.post('http://affiliation-matcher:5001/match_list', json=p)

In [11]:
r.json()

{'data': {'task_id': 'e445e83b-9b60-4be7-8b1a-6f3a834f0c7a'},
 'status': 'success'}

In [ ]:
#import timeout_decorator
import requests
params = {
    "match_types" : ["country"],
    "affiliations" : ["cnrs gsi university of bergen gsi helmholtzzentrum für schwerionenforschung bull-atos cern cnrs/in2p3 gsi helmholtzzentrum für schwerionenforschung gmbh gsi helmholtz centre for heavy ion research gmbh", "qinghai university affiliated hospital https://grid.ac/institutes/grid.459333.b grid zhejiang shuren university https://grid.ac/institutes/grid.413073.2 grid qinghai university https://grid.ac/institutes/grid.262246.6 grid sichuan university https://grid.ac/institutes/grid.13291.38 grid west china second university hospital of sichuan university https://grid.ac/institutes/grid.461863.e grid qinghai university https://grid.ac/institutes/grid.262246.6", "mit (massachusetts institute of technology) mit eaps australian national university,,mit (massachusetts institute of technology)mit eapsaustralian national university", "telespazio germany gmbhjulia computinguniversidad de la repúblicamassachusetts institute of technologyuniversity of oxforduniversidad nacional autónoma de méxico (unam)"
    ]
}
AFFILIATION_MATCHER_SERVICE='http://affiliation-matcher:5001'
matcher_endpoint_url = f'{AFFILIATION_MATCHER_SERVICE}/match_list'
r = requests.post(matcher_endpoint_url, json=params)
task_id = r.json()['data']['task_id']
print(f'New task {task_id} for matcher')
for i in range(0, 100000):
    r_task = requests.get(f'{AFFILIATION_MATCHER_SERVICE}/tasks/{task_id}').json()
    try:
        status = r_task['data']['task_status']
    except:
        print(f'Error in getting task {task_id} status : {r_task}')
        status = 'error'
    if status == 'finished':
        result = r_task['data']['task_result']
        break
    elif status in ['started', 'queued']:
        time.sleep(2)
        continue
    else:
        print(f'Error with task {task_id} : status {status}')
        print(f'{r_task}')
result

def get_matcher_parallel(affil_chunks):
    # prend une liste de liste d'affiliations
    #logger.debug(f'start parallel with {len(affil_chunks)} sublists')
    manager = mp.Manager()
    return_dict = manager.dict()
    jobs = []
    for ix, c in enumerate(affil_chunks):
        if len(c) == 0:
            continue
        p = mp.Process(target=get_matcher_results, args=(c, ix, return_dict))
        p.start()
        jobs.append(p)
    for p in jobs:
        p.join()
    #logger.debug(f'end parallel')
    flat_list = [item for sublist in return_dict.values() for item in sublist]
    return flat_list